In [1]:
import random

import numpy as np
import pandas as pd

import os
from os import listdir
from os.path import join, splitext

import cv2
from PIL import Image

import torch
from torch import nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Compose, ToTensor, ToPILImage

import matplotlib.pyplot as plt

Iterative Crowd Counting Model Libraries

In [2]:
from icc.data_loaderB import ImageDataLoader
from icc.model_ic_CNN import modelicCNN, retrain_icCNN
from icc.evaluate_icCNN import evaluate_model
from icc import network

GPU Configurations

In [3]:
# GPU to run on
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [4]:
# Fixing random seed
rand_seed = 26700

np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
torch.cuda.manual_seed(rand_seed)

Hyperparameters to tune

In [5]:
BATCH_SIZE = 12
MAX_EPOCH = 50

LR = 0.00001
WEIGHT_DECAY = 0.00001
MOMENTUM = 0.9

Data Loader Configuration

In [6]:
train_path = 'data/train/images'
train_gt_path = 'data/train/ground_truth_csv'

val_path = 'data/val/images'
val_gt_path = 'data/val/ground_truth_csv'

output_dir = 'logs/model_icCNN/'

In [7]:
train_data_loader = ImageDataLoader(train_path, 
                                    train_gt_path,
                                    shuffle=True,
                                    gt_downsample=False,
                                    pre_load=False,
                                    sr_mode=False)

val_data_loader = ImageDataLoader(val_path, 
                                  val_gt_path,
                                  shuffle=True,
                                  gt_downsample=False,
                                  pre_load=False,
                                  sr_mode=False)

Utils to read and transform data

In [8]:
def RandomCrop(Input, Density, h, w, th, tw):
    x1 = random.randint(0, h - th)
    y1 = random.randint(0, w - tw)

    Input = Input[x1:x1 + th, y1:y1 + tw]
    Density = Density.reshape((h, w))[x1:x1 + th, y1:y1 + tw]

    return Input, Density

In [9]:
def get_training_batch(blob):
    img = blob['data']
    gt_density = blob['gt_density']

    h = img.shape[0]
    w = img.shape[1]
    
    th = int(h/3.0 - ((h/3.0) % 4))
    tw = int(w/3.0 - ((w/3.0) % 4))
    
    Input_HR = torch.zeros(BATCH_SIZE, 1, th, tw)
    GT_Density = torch.zeros(BATCH_SIZE, 1, th, tw)

    for cur_step in range(0, BATCH_SIZE):
        img_crop, gt_density_crop = RandomCrop(img, gt_density, h, w, th, tw)

        Input_HR[cur_step] = torch.from_numpy(img_crop)
        GT_Density[cur_step] = torch.from_numpy(gt_density_crop)
        
    return Input_HR, GT_Density

Initializing ICC parameters

In [10]:
# Initializing model
net = modelicCNN()
net.cuda()

# Initializing optimizer
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, net.parameters()), lr=LR)
# optimizer = torch.optim.SGD(net.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, momentum=MOMENTUM)

# Initializing loss
n_loss_fn = nn.MSELoss()

Training Networks

In [11]:
best_maeHR = float('inf') #sys.maxint
best_epochHR = 1

N_Loss = []

In [12]:
for epoch in range(1, MAX_EPOCH+1):
    
    net.train()

    count = 0
    n_loss = 0
    for blob in train_data_loader:
        optimizer.zero_grad()

        Input_HR, GT_Density = get_training_batch(blob)
        
        Input_HR = Variable(Input_HR)
        GT_Density = Variable(GT_Density)
        
        Input_HR = Input_HR.cuda()
        GT_Density = GT_Density.cuda()
        
        Density = net(Input_HR)
        
        LossHR = n_loss_fn(Density, GT_Density)
        
        loss = (1000 * LossHR)
        loss.backward()
        optimizer.step()
                
        count += 1
        
        n_loss += loss.data[0]
        print("Training epoch: {}, Batch_Num: {}/{}, N_Loss: {}".format(
            epoch, count, train_data_loader.get_num_samples(), loss.data[0]))
        
    N_Loss.append(n_loss/count)
    
    if (epoch % 5 == 0):
        maeHR, mseHR = evaluate_model(net, val_data_loader)

        if maeHR < best_maeHR:
            best_maeHR = maeHR
            best_mseHR = mseHR
            best_epochHR = epoch

            network.save_net(os.path.join(output_dir, 'best_icCNN.h5'), net)

        print("EPOCH: %d, MAE_HR: %.1f, MSE_HR: %0.1f" % (epoch, maeHR, mseHR))
        print("BEST MAE_HR: %0.1f, BEST MSE_HR: %0.1f, BEST Epoch: %4.2f" % (best_maeHR, best_mseHR, best_epochHR))

/home/udbhav/thesis/brain-storming/Crowd_SRGAN_2/Crowd_SRGAN/icc/data_loaderB.py:103: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  den = pd.read_csv(os.path.join(self.gt_path,os.path.splitext(fname)[0] + '.csv'), sep=',',header=None).as_matrix()
/home/udbhav/anaconda3/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:183: UserWarning: nn.UpsamplingBilinear2d is deprecated. Use nn.Upsample instead.
  warnings.warn("nn.UpsamplingBilinear2d is deprecated. Use nn.Upsample instead.")


Training epoch: 1, Batch_Num: 1/300, N_Loss: 42.84021759033203
Training epoch: 1, Batch_Num: 2/300, N_Loss: 2.514537811279297
Training epoch: 1, Batch_Num: 3/300, N_Loss: 0.10253629833459854
Training epoch: 1, Batch_Num: 4/300, N_Loss: 0.036158062517642975
Training epoch: 1, Batch_Num: 5/300, N_Loss: 0.009580442681908607
Training epoch: 1, Batch_Num: 6/300, N_Loss: 0.0035459815990179777
Training epoch: 1, Batch_Num: 7/300, N_Loss: 0.003995529375970364
Training epoch: 1, Batch_Num: 8/300, N_Loss: 0.004674268886446953
Training epoch: 1, Batch_Num: 9/300, N_Loss: 0.003101392649114132
Training epoch: 1, Batch_Num: 10/300, N_Loss: 0.0024698420893400908
Training epoch: 1, Batch_Num: 11/300, N_Loss: 0.006208175793290138
Training epoch: 1, Batch_Num: 12/300, N_Loss: 0.0039164344780147076
Training epoch: 1, Batch_Num: 13/300, N_Loss: 0.016327088698744774
Training epoch: 1, Batch_Num: 14/300, N_Loss: 0.006103903986513615
Training epoch: 1, Batch_Num: 15/300, N_Loss: 0.008371256291866302
Training

Training epoch: 1, Batch_Num: 123/300, N_Loss: 0.015422440133988857
Training epoch: 1, Batch_Num: 124/300, N_Loss: 0.0019008954986929893
Training epoch: 1, Batch_Num: 125/300, N_Loss: 0.002296952996402979
Training epoch: 1, Batch_Num: 126/300, N_Loss: 0.0025151006411761045
Training epoch: 1, Batch_Num: 127/300, N_Loss: 0.022140365093946457
Training epoch: 1, Batch_Num: 128/300, N_Loss: 0.004444336984306574
Training epoch: 1, Batch_Num: 129/300, N_Loss: 0.0008258402231149375
Training epoch: 1, Batch_Num: 130/300, N_Loss: 0.0069687627255916595
Training epoch: 1, Batch_Num: 131/300, N_Loss: 0.000903665495570749
Training epoch: 1, Batch_Num: 132/300, N_Loss: 0.004090901464223862
Training epoch: 1, Batch_Num: 133/300, N_Loss: 0.014943767338991165
Training epoch: 1, Batch_Num: 134/300, N_Loss: 0.012375679798424244
Training epoch: 1, Batch_Num: 135/300, N_Loss: 0.008856630884110928
Training epoch: 1, Batch_Num: 136/300, N_Loss: 0.07888299226760864
Training epoch: 1, Batch_Num: 137/300, N_Loss

Training epoch: 1, Batch_Num: 244/300, N_Loss: 0.011933395639061928
Training epoch: 1, Batch_Num: 245/300, N_Loss: 0.00560403848066926
Training epoch: 1, Batch_Num: 246/300, N_Loss: 0.1339763104915619
Training epoch: 1, Batch_Num: 247/300, N_Loss: 0.006048221606761217
Training epoch: 1, Batch_Num: 248/300, N_Loss: 0.003178781596943736
Training epoch: 1, Batch_Num: 249/300, N_Loss: 0.007805480156093836
Training epoch: 1, Batch_Num: 250/300, N_Loss: 0.01098338607698679
Training epoch: 1, Batch_Num: 251/300, N_Loss: 0.012501158751547337
Training epoch: 1, Batch_Num: 252/300, N_Loss: 0.04049311950802803
Training epoch: 1, Batch_Num: 253/300, N_Loss: 0.001796227297745645
Training epoch: 1, Batch_Num: 254/300, N_Loss: 0.01105897780507803
Training epoch: 1, Batch_Num: 255/300, N_Loss: 0.0030630105175077915
Training epoch: 1, Batch_Num: 256/300, N_Loss: 0.044356703758239746
Training epoch: 1, Batch_Num: 257/300, N_Loss: 0.006553046405315399
Training epoch: 1, Batch_Num: 258/300, N_Loss: 0.0145

Training epoch: 2, Batch_Num: 66/300, N_Loss: 0.001087216311134398
Training epoch: 2, Batch_Num: 67/300, N_Loss: 0.0022157570347189903
Training epoch: 2, Batch_Num: 68/300, N_Loss: 0.006237523164600134
Training epoch: 2, Batch_Num: 69/300, N_Loss: 0.030269749462604523
Training epoch: 2, Batch_Num: 70/300, N_Loss: 0.023345094174146652
Training epoch: 2, Batch_Num: 71/300, N_Loss: 0.010570627637207508
Training epoch: 2, Batch_Num: 72/300, N_Loss: 0.004335569683462381
Training epoch: 2, Batch_Num: 73/300, N_Loss: 0.011113767512142658
Training epoch: 2, Batch_Num: 74/300, N_Loss: 0.008389235474169254
Training epoch: 2, Batch_Num: 75/300, N_Loss: 0.013818135485053062
Training epoch: 2, Batch_Num: 76/300, N_Loss: 0.019567513838410378
Training epoch: 2, Batch_Num: 77/300, N_Loss: 0.0068444060161709785
Training epoch: 2, Batch_Num: 78/300, N_Loss: 0.002373325638473034
Training epoch: 2, Batch_Num: 79/300, N_Loss: 0.009935220703482628
Training epoch: 2, Batch_Num: 80/300, N_Loss: 0.078568845987

...

Training epoch: 9, Batch_Num: 276/300, N_Loss: 0.001663168310187757
Training epoch: 9, Batch_Num: 277/300, N_Loss: 0.011832408607006073
Training epoch: 9, Batch_Num: 278/300, N_Loss: 0.03828295320272446
Training epoch: 9, Batch_Num: 279/300, N_Loss: 0.002622415544465184
Training epoch: 9, Batch_Num: 280/300, N_Loss: 0.004725074861198664
Training epoch: 9, Batch_Num: 281/300, N_Loss: 0.02391239069402218
Training epoch: 9, Batch_Num: 282/300, N_Loss: 0.0030705241952091455
Training epoch: 9, Batch_Num: 283/300, N_Loss: 0.007833058945834637
Training epoch: 9, Batch_Num: 284/300, N_Loss: 0.0013192167971283197
Training epoch: 9, Batch_Num: 285/300, N_Loss: 0.017305172979831696
Training epoch: 9, Batch_Num: 286/300, N_Loss: 0.014025083743035793
Training epoch: 9, Batch_Num: 287/300, N_Loss: 0.008709823712706566
Training epoch: 9, Batch_Num: 288/300, N_Loss: 0.011650084517896175
Training epoch: 9, Batch_Num: 289/300, N_Loss: 0.002316212747246027
Training epoch: 9, Batch_Num: 290/300, N_Loss: 0

Training epoch: 10, Batch_Num: 97/300, N_Loss: 0.012839465402066708
Training epoch: 10, Batch_Num: 98/300, N_Loss: 0.014457613229751587
Training epoch: 10, Batch_Num: 99/300, N_Loss: 0.024677036330103874
Training epoch: 10, Batch_Num: 100/300, N_Loss: 0.05791085958480835
Training epoch: 10, Batch_Num: 101/300, N_Loss: 0.010448056273162365
Training epoch: 10, Batch_Num: 102/300, N_Loss: 0.011962159536778927
Training epoch: 10, Batch_Num: 103/300, N_Loss: 0.03652031719684601
Training epoch: 10, Batch_Num: 104/300, N_Loss: 0.005754416808485985
Training epoch: 10, Batch_Num: 105/300, N_Loss: 0.004741739481687546
Training epoch: 10, Batch_Num: 106/300, N_Loss: 0.0018087899079546332
Training epoch: 10, Batch_Num: 107/300, N_Loss: 0.009316042065620422
Training epoch: 10, Batch_Num: 108/300, N_Loss: 0.005830989684909582
Training epoch: 10, Batch_Num: 109/300, N_Loss: 0.02702775038778782
Training epoch: 10, Batch_Num: 110/300, N_Loss: 0.0014961343258619308
Training epoch: 10, Batch_Num: 111/300

Training epoch: 10, Batch_Num: 216/300, N_Loss: 0.011255210265517235
Training epoch: 10, Batch_Num: 217/300, N_Loss: 0.0024418605025857687
Training epoch: 10, Batch_Num: 218/300, N_Loss: 0.013278383761644363
Training epoch: 10, Batch_Num: 219/300, N_Loss: 0.0020010103471577168
Training epoch: 10, Batch_Num: 220/300, N_Loss: 0.00661398284137249
Training epoch: 10, Batch_Num: 221/300, N_Loss: 0.026382341980934143
Training epoch: 10, Batch_Num: 222/300, N_Loss: 0.0015756174689158797
Training epoch: 10, Batch_Num: 223/300, N_Loss: 0.010943284258246422
Training epoch: 10, Batch_Num: 224/300, N_Loss: 0.0014773744624108076
Training epoch: 10, Batch_Num: 225/300, N_Loss: 0.0018126191571354866
Training epoch: 10, Batch_Num: 226/300, N_Loss: 0.003275549039244652
Training epoch: 10, Batch_Num: 227/300, N_Loss: 0.0018217972246930003
Training epoch: 10, Batch_Num: 228/300, N_Loss: 0.003990738186985254
Training epoch: 10, Batch_Num: 229/300, N_Loss: 0.019055599346756935
Training epoch: 10, Batch_Num

Training epoch: 11, Batch_Num: 34/300, N_Loss: 0.010308513417840004
Training epoch: 11, Batch_Num: 35/300, N_Loss: 0.01582232676446438
Training epoch: 11, Batch_Num: 36/300, N_Loss: 0.003496772376820445
Training epoch: 11, Batch_Num: 37/300, N_Loss: 0.00624769227579236
Training epoch: 11, Batch_Num: 38/300, N_Loss: 0.03600194677710533
Training epoch: 11, Batch_Num: 39/300, N_Loss: 0.002838348736986518
Training epoch: 11, Batch_Num: 40/300, N_Loss: 0.0025606947019696236
Training epoch: 11, Batch_Num: 41/300, N_Loss: 0.0114286495372653
Training epoch: 11, Batch_Num: 42/300, N_Loss: 0.1402272880077362
Training epoch: 11, Batch_Num: 43/300, N_Loss: 0.011304483748972416
Training epoch: 11, Batch_Num: 44/300, N_Loss: 0.002988188061863184
Training epoch: 11, Batch_Num: 45/300, N_Loss: 0.003172166645526886
Training epoch: 11, Batch_Num: 46/300, N_Loss: 0.12925352156162262
Training epoch: 11, Batch_Num: 47/300, N_Loss: 0.04169008508324623
Training epoch: 11, Batch_Num: 48/300, N_Loss: 0.0057671

KeyboardInterrupt: 

In [ ]:
network.save_net(os.path.join(output_dir, 'last_epoch_icCNN.h5'), net)

In [ ]:
print(N_Loss)